In [70]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-driver

In [71]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("UDF").getOrCreate()

In [72]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType


In [78]:
from google.colab import files

uploaded = files.upload()

Saving NFL_PRODAY.csv to NFL_PRODAY (8).csv


In [109]:
import pandas as pd
import io
  
df = pd.read_csv('NFL_PRODAY.csv', header= 0,
                        encoding= 'unicode_escape')
df.head()

,ï»¿Year,ACC,Big Ten,Big 12,Pac-12,SEC,Other,Height_IN,Weight_LBS,Wonderlic,40_Yard,Bench_Press,Vert LeapÂ (in),Broad JumpÂ (in),Shuttle,3Cone,Started
0,2021,1,0,0,0,0,1,72.00,211,NaN,4.65,NaN,32.5,115,4.20,7.00,No
1,2021,0,0,0,0,0,0,72.25,210,NaN,4.96,NaN,30.0,114,4.60,7.34,No
2,2021,0,0,0,0,0,0,76.50,225,NaN,5.11,NaN,27.5,103,4.75,NaN,No
3,2021,0,0,1,0,1,0,73.13,220,NaN,4.84,NaN,36.5,114,4.44,7.15,No
4,2021,0,1,0,0,1,0,74.75,227,NaN,4.46,NaN,NaN,119,NaN,NaN,Yes


In [110]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ï»¿Year           70 non-null     int64  
 1   ACC               70 non-null     int64  
 2   Big Ten           70 non-null     int64  
 3   Big 12            70 non-null     int64  
 4   Pac-12            70 non-null     int64  
 5   SEC               70 non-null     int64  
 6   Other             70 non-null     int64  
 7   Height_IN         70 non-null     float64
 8   Weight_LBS        70 non-null     int64  
 9   Wonderlic         0 non-null      float64
 10  40_Yard           66 non-null     float64
 11  Bench_Press       6 non-null      float64
 12  Vert LeapÂ (in)   65 non-null     float64
 13  Broad JumpÂ (in)  70 non-null     int64  
 14  Shuttle           64 non-null     float64
 15  3Cone             63 non-null     float64
 16  Started           70 non-null     object 
dtyp

In [111]:
df.drop(columns=["Wonderlic","Bench_Press"],axis=1, inplace=True)

In [115]:
df.drop(columns="ï»¿Year", "Big Ten", "Big 12", "SEC", "Other","Pac-12")

SyntaxError: ignored

In [116]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63 entries, 0 to 69
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ï»¿Year           63 non-null     int64  
 1   ACC               63 non-null     int64  
 2   Big Ten           63 non-null     int64  
 3   Big 12            63 non-null     int64  
 4   Pac-12            63 non-null     int64  
 5   SEC               63 non-null     int64  
 6   Other             63 non-null     int64  
 7   Height_IN         63 non-null     float64
 8   Weight_LBS        63 non-null     int64  
 9   40_Yard           63 non-null     float64
 10  Vert LeapÂ (in)   63 non-null     float64
 11  Broad JumpÂ (in)  63 non-null     int64  
 12  Shuttle           63 non-null     float64
 13  3Cone             63 non-null     float64
 14  Started           63 non-null     object 
dtypes: float64(5), int64(9), object(1)
memory usage: 7.9+ KB


In [117]:
df.drop(columns=['ï»¿Year'],axis=1, inplace=True)

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63 entries, 0 to 69
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ACC               63 non-null     int64  
 1   Big Ten           63 non-null     int64  
 2   Big 12            63 non-null     int64  
 3   Pac-12            63 non-null     int64  
 4   SEC               63 non-null     int64  
 5   Other             63 non-null     int64  
 6   Height_IN         63 non-null     float64
 7   Weight_LBS        63 non-null     int64  
 8   40_Yard           63 non-null     float64
 9   Vert LeapÂ (in)   63 non-null     float64
 10  Broad JumpÂ (in)  63 non-null     int64  
 11  Shuttle           63 non-null     float64
 12  3Cone             63 non-null     float64
 13  Started           63 non-null     object 
dtypes: float64(5), int64(8), object(1)
memory usage: 7.4+ KB


In [119]:
X = pd.get_dummies(df)
X

,ACC,Big Ten,Big 12,Pac-12,SEC,Other,Height_IN,Weight_LBS,40_Yard,Vert LeapÂ (in),Broad JumpÂ (in),Shuttle,3Cone,Started_No,Started_Yes
0,1,0,0,0,0,1,72.00,211,4.65,32.5,115,4.20,7.00,1,0
1,0,0,0,0,0,0,72.25,210,4.96,30.0,114,4.60,7.34,1,0
3,0,0,1,0,1,0,73.13,220,4.84,36.5,114,4.44,7.15,1,0
5,0,0,0,0,0,0,78.63,234,4.61,32.5,117,4.22,7.16,1,0
10,0,1,0,0,0,0,73.88,215,4.77,28.0,108,4.41,7.28,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0,1,0,0,0,1,75.00,225,5.13,28.5,104,4.59,7.65,1,0
66,0,0,0,0,0,1,77.13,237,4.87,29.0,111,4.24,6.97,1,0
67,0,0,0,0,0,1,73.88,223,4.78,29.0,105,4.40,7.11,0,1
68,0,0,0,0,0,1,74.38,219,4.99,29.0,105,4.46,7.46,1,0


In [120]:
X.drop(columns=["Started_No"],axis=1, inplace=True)

In [121]:
X.head()

,ACC,Big Ten,Big 12,Pac-12,SEC,Other,Height_IN,Weight_LBS,40_Yard,Vert LeapÂ (in),Broad JumpÂ (in),Shuttle,3Cone,Started_Yes
0,1,0,0,0,0,1,72.00,211,4.65,32.5,115,4.20,7.00,0
1,0,0,0,0,0,0,72.25,210,4.96,30.0,114,4.60,7.34,0
3,0,0,1,0,1,0,73.13,220,4.84,36.5,114,4.44,7.15,0
5,0,0,0,0,0,0,78.63,234,4.61,32.5,117,4.22,7.16,0
10,0,1,0,0,0,0,73.88,215,4.77,28.0,108,4.41,7.28,0


In [122]:
y=X["Started_Yes"]
X.drop(columns=["Started_Yes"],axis=1, inplace=True)

In [123]:
from sklearn.model_selection import train_test_split

In [124]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [125]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [126]:
classifier.fit(X_train, y_train)
print(classifier)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [127]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.9361702127659575
Testing Data Score: 0.875


In [128]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [129]:
classifier.fit(X_train, y_train)
print(classifier)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [130]:
def getTestData(ACC, Big_Ten, Big_12,	Pac_12, SEC, Other,	Height_IN,	Weight_LBS,	40_Yard,	Vert_Leap_IN,	Broad_Jump_IN,	Shuttle,	3Cone):
  testData = [ACC, Big_Ten, Big_12,Pac_12, SEC, Other,	Height_IN,	Weight_LBS,	40_Yard,	Vert_Leap_IN,	Broad_Jump_IN,	Shuttle,	3Cone]
  return testData

SyntaxError: ignored

In [131]:
new_data=[1,	0	,0,	0,	0	,1,	72.00,	211,	4.65,	32.5,	115,	4.20,	7.00	]

In [132]:
import numpy as np
testData=np.reshape(new_data,(1,-1))

In [133]:
predictions = classifier.predict(testData)
print("Players are either 0 (not a starter) or 1 (stared)")
print(f"The player was classified as: {predictions}")

Players are either 0 (not a starter) or 1 (stared)
The player was classified as: [0]
